In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Fixed parameters (number of samples)

In [2]:
#Number of samples determined by Chebyshev inequality
samples = 10000

# Variable parameters (qubits, layers and distributions)

In [3]:
#List of distributions
distributions = ['uniform', 'normal', 'exponential']

#List of number of qubits
qubits = [3]

#List of number of layers
layers = [6,7,8,9,10]

# Function to calculate Meyer-Wallach mean and standard deviation

In [4]:
def ent_cap(dist,samples, circuit, nlayers):
    MW_mean = 0
    MW_mean_sqr = 0
    
    
    if dist == 'normal':
        
        for i in range(samples):
            angles = np.array([ [[np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)], 
                                [np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
            
            
            pur_sum = 0
            for k in range(wires):
                state = circuit(angles, k)
                pur_sum = pur_sum + np.trace(np.matmul(state, state))

            MW_measure = 2*(1-1/wires*pur_sum)
            MW_measure_sqr = MW_measure**2

            MW_mean = MW_mean + MW_measure
            MW_mean_sqr = MW_mean_sqr + MW_measure_sqr
            
            
    if dist == 'exponential':
        
        for i in range(samples):
            angles = np.array([ [[np.random.exponential(scale = 0.865) for i in range(wires)], 
                                [np.random.exponential(scale = 0.865) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
            
            pur_sum = 0
            for k in range(wires):
                state = circuit(angles, k)
                pur_sum = pur_sum + np.trace(np.matmul(state, state))

            MW_measure = 2*(1-1/wires*pur_sum)
            MW_measure_sqr = MW_measure**2

            MW_mean = MW_mean + MW_measure
            MW_mean_sqr = MW_mean_sqr + MW_measure_sqr
            
            
    if dist == 'uniform':
        
        for i in range(samples):
            angles = np.array([ [[np.random.uniform(low=0, high=2*np.pi) for i in range(wires)], 
                                [np.random.uniform(low=0, high=2*np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
            
            pur_sum = 0
            for k in range(wires):
                state = circuit(angles, k)
                pur_sum = pur_sum + np.trace(np.matmul(state, state))

            MW_measure = 2*(1-1/wires*pur_sum)
            MW_measure_sqr = MW_measure**2

            MW_mean = MW_mean + MW_measure
            MW_mean_sqr = MW_mean_sqr + MW_measure_sqr
    
    
    
    MW_mean = MW_mean/samples
    MW_mean_sqr = MW_mean_sqr/samples
    MW_standev = np.sqrt(MW_mean_sqr - MW_mean**2)
    return(MW_mean, MW_standev)

# Code that executes the different conditions

In [5]:
now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")
print("Tempo inicial: ")
print(now0)
print()


for dist in distributions:
    
    for wires in qubits:
        dev = qml.device("default.qubit", wires=wires, shots=None)    
    
        for nlayers in layers:
            
            #No connections circuit
            def layer_noconnec(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])


            def noconnec(rotations, k):
                qml.layer(layer_noconnec, nlayers, rotations)
                return qml.density_matrix([k])

            noconnec_circuit = qml.QNode(noconnec, dev)



            #Linear circuit
            def layer_linear(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")   

            def linear(rotations, k):
                qml.layer(layer_linear, nlayers, rotations)
                return qml.density_matrix([k])

            linear_circuit = qml.QNode(linear, dev)



            #Ring circuit
            def layer_ring(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
                qml.CNOT(wires=[wires-1,0]) 

            def ring(rotations, k):
                qml.layer(layer_ring, nlayers, rotations)
                return qml.density_matrix([k])

            ring_circuit = qml.QNode(ring, dev)        


            ent_noconnec, ent_noconnec_standev = ent_cap(dist, samples, noconnec_circuit, nlayers)
            ent_linear, ent_linear_standev = ent_cap(dist, samples, linear_circuit, nlayers)
            ent_ring, ent_ring_standev = ent_cap(dist, samples, ring_circuit, nlayers)

            f = open("entanglement_ansatz1_dist{0}_qubits{1}_layers{2}_samples{3}.txt"
                         .format(dist,wires,nlayers,samples), "w")

            print('Circuit,MW_mean,MW_standdev', file = f)
            print('No connections,',ent_noconnec,',', ent_noconnec_standev, file = f)
            print('Linear,',ent_linear,',', ent_linear_standev, file = f)
            print('Ring,',ent_ring,',', ent_ring_standev, file = f)

            print('Execution: distribution=',dist,'; qubits=',wires,'; layers=', nlayers)
            now1 = dt.datetime.now() #monitorar tempo de início
            now1 = now1.strftime("%Y-%m-%d %H:%M:%S")
            print("Tempo: ",now1)
            print()
     
    
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

f.close()

print()
print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Tempo inicial: 
2023-09-29 15:18:48

Execution: distribution= uniform ; qubits= 3 ; layers= 1
Tempo:  2023-09-29 15:36:42

Execution: distribution= uniform ; qubits= 3 ; layers= 2
Tempo:  2023-09-29 16:09:31

Execution: distribution= uniform ; qubits= 3 ; layers= 3
Tempo:  2023-09-29 16:55:32

Execution: distribution= uniform ; qubits= 3 ; layers= 4
Tempo:  2023-09-29 17:56:49

Execution: distribution= uniform ; qubits= 3 ; layers= 5
Tempo:  2023-09-29 19:11:53

Execution: distribution= normal ; qubits= 3 ; layers= 1
Tempo:  2023-09-29 19:28:55

Execution: distribution= normal ; qubits= 3 ; layers= 2
Tempo:  2023-09-29 19:58:23

Execution: distribution= normal ; qubits= 3 ; layers= 3
Tempo:  2023-09-29 20:40:55

Execution: distribution= normal ; qubits= 3 ; layers= 4
Tempo:  2023-09-29 21:37:15

Execution: distribution= normal ; qubits= 3 ; layers= 5
Tempo:  2023-09-29 22:44:41

Execution: distribution= exponential ; qubits= 3 ; layers= 1
Tempo:  2023-09-29 23:00:47

Execution: distrib